In [ ]:
import numpy as np
import random
import pandas as pd
import os
import matplotlib.pyplot as plt
import pathlib
import shutil
import datetime
import pickle
from collections import Counter
import pydicom
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
import tensorflow.keras.layers as KL
from tensorflow.keras import Model
from tensorflow.keras.mixed_precision import experimental as mixed_precision

import efficientnet.tfkeras as efn # https://github.com/qubvel/efficientnet

AUTOTUNE = tf.data.experimental.AUTOTUNE

# Check GPUs:",
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            # Prevent TensorFlow from allocating all memory of all GPUs:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
        
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)
#print('Compute dtype: %s' % policy.compute_dtype)
#print('Variable dtype: %s' % policy.variable_dtype)

In [ ]:
JPEG_TRAIN = 'data/siim-isic-melanoma-classification/jpeg/train'
JPEG_VAL = 'data/siim-isic-melanoma-classification/jpeg/val'
JPEG_TEST = 'data/siim-isic-melanoma-classification/jpeg/test'
DCM_TRAIN = 'data/siim-isic-melanoma-classification/train'
DCM_VAL = 'data/siim-isic-melanoma-classification/val'
DCM_TEST = 'data/siim-isic-melanoma-classification/test'
CSV_TRAIN = 'data/siim-isic-melanoma-classification/train.csv'
CSV_TEST = 'data/siim-isic-melanoma-classification/test.csv'
SUBMITS_DIR = 'submits/siim-isic-melanoma-classification'

CACHE_FILE = 'cache/siim-isic-melanoma-classification/cache'

N_TEST = 10982

WIDTH_PRE, HEIGHT_PRE = 768, 768

WIDTH_0, HEIGHT_0 = 224, 224
WIDTH_4, HEIGHT_4 = 380, 380
#WIDTH_5, HEIGHT_5 = 456, 456
#WIDTH_7, HEIGHT_7 = 600, 600
#WIDTH_7, HEIGHT_7 = 768, 768
#WIDTH, HEIGHT = 448, 448
#WIDTH, HEIGHT = 896, 896
#WIDTH, HEIGHT = 768, 768
#WIDTH, HEIGHT = 1024, 1024
#WIDTH, HEIGHT = 1120, 1120

WIDTH, HEIGHT = WIDTH_0, HEIGHT_0
WIDTH, HEIGHT = WIDTH_4, HEIGHT_4


# AUGMENTATIONS:
FLIP_ROT = True
BR_SAT_CON = False
CROP = False

TEST_AUGMENT_N = 16

BATCH_SIZE = 8

METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc')
]

BODY_PARTS_ONE_HOT = {'HEAD/NECK':       [1,0,0,0,0,0,0], 
                      'LOWER EXTREMITY': [0,1,0,0,0,0,0],
                      'ORAL/GENITAL':    [0,0,1,0,0,0,0],
                      'PALMS/SOLES':     [0,0,0,1,0,0,0],
                      'TORSO':           [0,0,0,0,1,0,0],
                      'UPPER EXTREMITY': [0,0,0,0,0,1,0], 
                      'SKIN':            [0,0,0,0,0,0,1]
                     }
PATIENT_DATA_LEN = 9

In [ ]:
meta = pd.read_csv(CSV_TRAIN)
meta_test = pd.read_csv(CSV_TEST)
meta_test

In [ ]:
def split_train_val(meta_df, train_path, val_path, val_percent = 0.25, seed = 18, file_ext = 'jpg'):
    n = len(meta_df)
    n_val = int(n * val_percent)
    n_train = n - n_val
    train_val = ['train'] * n_train + ['val'] * n_val
    random.seed(seed)
    random.shuffle(train_val)
    meta_df['train_val'] = train_val
    
    p = pathlib.Path(train_path)
    l = list(p.glob(str('**/*.' + file_ext)))
    if len(l) != n:
        print('Is train/val split already done?')
        return meta_df
    
    i = 0
    for source in l:
        #print(source)
        if meta_df.loc[meta_df['image_name'] == source.stem, 'train_val'].values[0] == 'val':
            #print('val', source.stem)
            dest = pathlib.Path(val_path, source.name)
            #print(dest)
            shutil.move(source, dest)
            i += 1
    
    if i == n_val:
        print(n_val, 'validation images moved to validation directory')
    else:
        print('There is a discrepancy in number of validation images moved')
        print('Images supposed to be moved:', n_val)
        print('Images moved:', i)
    
    return meta_df

In [ ]:
#meta = split_train_val(meta, JPEG_TRAIN, JPEG_VAL, file_ext = 'jpg')
meta = split_train_val(meta, DCM_TRAIN, DCM_VAL, file_ext = 'dcm')
meta

In [ ]:
def move_to_target_subfolders(meta_df, data_path, target_list, file_ext = 'jpg'):
    
    p = pathlib.Path(data_path)
    for target in target_list:
        p_target = p.joinpath(str(target))
        p_target.mkdir(exist_ok = True)
    
    l = list(p.glob(str('**/*.' + file_ext)))
    
    for p_image in l:
        image_name = p_image.stem
        #print(image_name)
        target = meta_df.loc[meta_df['image_name'] == image_name, 'target'].values[0]
        #print(target)
        p_target = p.joinpath(str(target)).joinpath(p_image.name)
        shutil.move(p_image, p_target)
        #print(p_target)
        #break
    print('Images moved to target subfolders')
        
#move_to_target_subfolders(meta, JPEG_TRAIN, [0, 1], file_ext = 'jpg')
#move_to_target_subfolders(meta, JPEG_VAL, [0, 1], file_ext = 'jpg')
#move_to_target_subfolders(meta, DCM_TRAIN, [0, 1], file_ext = 'dcm')
#move_to_target_subfolders(meta, DCM_VAL, [0, 1], file_ext = 'dcm')

In [ ]:
ds = pydicom.dcmread(str(DCM_TRAIN + '/0/ISIC_9999320.dcm'))
age = int(ds.PatientAge[:-1]) / 100
sex = ds.PatientSex
if sex == 'M':
    sex = 1
else:
    sex = 0
body_part = np.array(BODY_PARTS_ONE_HOT[ds.BodyPartExamined])
patient_info = np.concatenate([np.array([age, sex]), body_part])
patient_info = tf.convert_to_tensor(patient_info, dtype = tf.float32)
patient_info

In [ ]:
def augment(img_patient_data, label):
    img, patient_data = img_patient_data
    #if CROP:
    #    crop_factor = tf.random.uniform(shape = (), minval = 1, maxval = 1.5)
    #    #tf.print(crop_factor)
    #    img = preprocess_images(img, int(HEIGHT*crop_factor), int(WIDTH*crop_factor))
    #    img = tf.image.random_crop(img, [HEIGHT, WIDTH, 3])
        
    if FLIP_ROT:
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        k = tf.random.uniform(shape = (), minval=0, maxval=4, dtype=tf.int32)
        img = tf.image.rot90(img, k)
        
    if BR_SAT_CON:
        img = tf.image.random_brightness(img, max_delta=63. / 255.)
        img = tf.image.random_saturation(img, lower=0.5, upper=1.5)
        img = tf.image.random_contrast(img, lower=0.2, upper=1.8)
    return (img, patient_data), label

def decode_dcm_img(img_path):
    img_path = str(img_path.numpy(), 'utf-8')
    ds = pydicom.dcmread(img_path)
    img = ds.pixel_array
    img = pydicom.pixel_data_handlers.util.convert_color_space(img, 'YBR_FULL_422', 'RGB')
    img = tf.convert_to_tensor(img, dtype = tf.uint8)
    return img

def resize(img_patient_data, label):
    img, patient_data = img_patient_data
    #img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [HEIGHT, WIDTH])
    return (img, patient_data), label

def resize_with_crop(img_patient_data, label):
    img, patient_data = img_patient_data
    #img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize_with_crop_or_pad(img, HEIGHT, WIDTH)
    return (img, patient_data), label

def resize_with_pad(img_patient_data, label):
    img, patient_data = img_patient_data
    #img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize_with_pad(img, HEIGHT, WIDTH)
    return (img, patient_data), label
    
def decode_dcm_patient_data(img_path):
    img_path = str(img_path.numpy(), 'utf-8')
    ds = pydicom.dcmread(img_path)
    
    # Patient meta
    age = int(ds.PatientAge[:-1]) / 100
    sex = ds.PatientSex
    if sex == 'M':
        sex = 1
    else:
        sex = 0
    body_part = np.array(BODY_PARTS_ONE_HOT[ds.BodyPartExamined])
    patient_data = np.concatenate([np.array([age, sex]), body_part])
    patient_data = tf.convert_to_tensor(patient_data, dtype = tf.float32)
    
    return patient_data
    
def process_path(img_path):
    #image_name = tf.strings.split(tf.strings.split(img_path, os.sep)[-1], '.')[0]
    #ext = tf.strings.split(tf.strings.split(img_path, os.sep)[-1], '.')[1]
    #tf.print(image_name)
    train_val_test = tf.strings.split(img_path, os.sep)[2]
    #tf.print(train_val_test)

    if train_val_test != tf.constant('test', dtype = tf.string):
        label = tf.strings.split(img_path, os.sep)[-2]
        label = tf.strings.to_number(label, tf.int32) 
    else:
        label = -1
    
    img = tf.py_function(func = decode_dcm_img, inp = [img_path], Tout = tf.uint8)
    img.set_shape([None, None, 3])

    patient_data = tf.py_function(func = decode_dcm_patient_data, inp = [img_path], Tout = tf.float32)
    patient_data.set_shape([PATIENT_DATA_LEN])
    #tf.print('patient_data', patient_data)
    
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [HEIGHT_PRE, WIDTH_PRE])
    #img = tf.image.resize_with_pad(img, HEIGHT_PRE, WIDTH_PRE)
    
    #tf.print(img.shape, patient_data.shape, label.shape)
    return (img, patient_data), label

def dataset_from_image_files(img_path, shuffle_buffer_size, cache = False, ext = 'dcm', test_augment = False):
    train_val_test = img_path.split('/')[2]
    if img_path.split('/')[-1] != train_val_test:
        target = img_path.split('/')[-1]
    else:
        target = None
    
    if train_val_test == 'test':
        ds = tf.data.Dataset.list_files(str(img_path + '*.' + ext), shuffle = False)
    else:
        ds = tf.data.Dataset.list_files(str(img_path + '*.' + ext), shuffle = True)
    ds = ds.map(process_path, num_parallel_calls=AUTOTUNE)
    
    if isinstance(cache, str):
        ds = ds.cache(str(cache + '-pre-' + str(HEIGHT_PRE) + 'x' + str(WIDTH_PRE)))
    
    ds = ds.map(resize, num_parallel_calls=AUTOTUNE)
    #ds = ds.map(resize_with_crop, num_parallel_calls=AUTOTUNE)
    #ds = ds.map(resize_with_pad, num_parallel_calls=AUTOTUNE)
    
    if isinstance(cache, str):
        ds = ds.cache(str(cache + '-' + str(HEIGHT) + 'x' + str(WIDTH)))
        
    if train_val_test != 'test':
        ds = ds.shuffle(buffer_size=shuffle_buffer_size)
        ds = ds.repeat()

    if test_augment:
        ds = ds.repeat(TEST_AUGMENT_N)
        ds = ds.map(augment, num_parallel_calls=AUTOTUNE)
        
    if target == '1' and train_val_test == 'train':
        tf.print('augmenting', img_path)
        ds = ds.map(augment, num_parallel_calls=AUTOTUNE)
        
    if target == None:
        ds = ds.batch(BATCH_SIZE)
    
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [ ]:
def dataset_combine(img_path, cache, shuffle_buffer_size = 1000):
    ds_0 = dataset_from_image_files(str(img_path + '/0'),
                                    shuffle_buffer_size = shuffle_buffer_size, 
                                    cache = str(cache + '-0'), ext = 'dcm')
    ds_1 = dataset_from_image_files(str(img_path + '/1'), 
                                    shuffle_buffer_size = shuffle_buffer_size, 
                                    cache = str(cache + '-1'), ext = 'dcm')

    w = 0.8
    ds = tf.data.experimental.sample_from_datasets([ds_0, ds_1], weights=[w, 1-w])
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds


ds_train = dataset_combine(DCM_TRAIN, cache = str(CACHE_FILE + '-train'), 
                           shuffle_buffer_size = 1000)

ds_val = dataset_from_image_files(DCM_VAL, shuffle_buffer_size = 1000, 
                                  cache = str(CACHE_FILE + '-val'), ext = 'dcm')

ds_test = dataset_from_image_files(DCM_TEST, shuffle_buffer_size = 100, 
                                  cache = str(CACHE_FILE + '-test'), ext = 'dcm')

ds_test_augment = dataset_from_image_files(DCM_TEST, shuffle_buffer_size = 100, 
                                           cache = str(CACHE_FILE + '-test'), ext = 'dcm', 
                                           test_augment = True)

In [ ]:
def show(image, patient_data, label):
    for i in range(BATCH_SIZE):
        plt.figure()
        plt.imshow(image[i,:,:,:])
        plt.title(str(i) + ' - ' + str(label.numpy()[i]) + str(patient_data.numpy()[i]))
        plt.axis('off')

In [ ]:
for (image, patient_data), label in ds_test_augment.take(1):
    show(image, patient_data, label)
    print(label.shape)
    print(image.shape)
    print(image.dtype)

In [ ]:
def build_efficientnet_bX(level = 0, lr = 0.001, compile_model = True, last_layer = True, 
                          height = HEIGHT, width = WIDTH):
    if level == 0:
        base_model = efn.EfficientNetB0(include_top=True, weights='noisy-student',
                                   input_shape=(height, width, 3))
    elif level == 1:
        base_model = efn.EfficientNetB1(include_top=True, weights='noisy-student',
                                   input_shape=(height, width, 3))
    elif level == 2:
        base_model = efn.EfficientNetB2(include_top=True, weights='noisy-student',
                                   input_shape=(height, width, 3))
    elif level == 3:
        base_model = efn.EfficientNetB3(include_top=True, weights='noisy-student',
                                   input_shape=(height, width, 3))
    elif level == 4:
        base_model = efn.EfficientNetB4(include_top=True, weights='noisy-student',
                                   input_shape=(height, width, 3))
    elif level == 5:
        base_model = efn.EfficientNetB5(include_top=True, weights='noisy-student',
                                   input_shape=(height, width, 3))
    elif level == 6:
        base_model = efn.EfficientNetB6(include_top=True, weights='noisy-student',
                                   input_shape=(height, width, 3))
    elif level == 7:
        base_model = efn.EfficientNetB7(include_top=True, weights='noisy-student',
                                   input_shape=(height, width, 3))
    
    patient_data_input = KL.Input(shape = (PATIENT_DATA_LEN,), name = 'patient_data_input')
    new_effnet_output = KL.Dense(7, activation = 'relu')(base_model.layers[-2].output)
    x = KL.concatenate([new_effnet_output, patient_data_input])
    x = KL.Dropout(0.5)(x)
    x = KL.Dense(8, activation = 'relu')(x)
    x = KL.Dropout(0.5)(x)
    x = KL.Dense(16, activation = 'relu')(x)
    x = KL.Dropout(0.5)(x)
    x = KL.Dense(8, activation = 'relu')(x)
    x = KL.Dropout(0.5)(x)
    if last_layer:
        x = KL.Dense(1, activation = 'sigmoid')(x)
        
    model = Model((base_model.input, patient_data_input), x)
    
    if compile_model:
        opt = tf.keras.optimizers.Adam(learning_rate = lr)
        model.compile(loss='binary_crossentropy',
                      optimizer=opt,
                      metrics=METRICS)
    model.summary()
    return model
model_0 = build_efficientnet_bX(level = 0, compile_model = True, last_layer = True, 
                                lr = 0.0005, height = HEIGHT_0, width = WIDTH_0)
model_4 = build_efficientnet_bX(level = 4, compile_model = True, last_layer = True, 
                                lr = 0.00001, height = HEIGHT_4, width = WIDTH_4)
#model_4_crop = build_efficientnet_bX(level = 4, compile_model = True, last_layer = True, 
#                                     lr = 0.001, height = HEIGHT_4, width = WIDTH_4)
#model_5 = build_efficientnet_bX(level = 5, compile_model = True, last_layer = True, 
#                                lr = 0.001, height = HEIGHT_5, width = WIDTH_5)

In [ ]:
def tb_callback(model_name):
    log_dir = pathlib.Path('logs/siim-isic-melanoma-classification/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath = str('models/siim-isic-melanoma-classification/'+model_name+'-{epoch:02d}-{val_auc:.4f}.h5'), 
        monitor = "val_auc",
        mode='max',
        save_best_only = False,
        save_weights_only = True,
    )
    return [tensorboard_callback, checkpoint_callback]
callbacks_0 = tb_callback(str('effnet-b0-' + str(WIDTH_0)))
callbacks_4 = tb_callback(str('effnet-b4-' + str(WIDTH_4)))
callbacks_4_crop = tb_callback(str('effnet-b4-crop' + str(WIDTH_4)))
#callbacks_5 = tb_callback(str('effnet-b5-' + str(WIDTH_5)))

In [ ]:
model_0.load_weights('models/siim-isic-melanoma-classification/effnet-b0-224-11-0.8551.h5')
model_4.load_weights('models/siim-isic-melanoma-classification/effnet-b4-380-01-0.8948.h5')

In [ ]:
history = model_4.fit(ds_train, 
                      validation_data = ds_val,
                      epochs = 2, 
                      steps_per_epoch = 1500, 
                      validation_steps = 1500, 
                      callbacks = callbacks_4)

In [ ]:
model.evaluate(ds_val, steps = 2000)

In [ ]:
#model.save_weights('models/siim-isic-melanoma-classification/2.h5')

In [ ]:
def predict(model, img_path, dataset, csv_filename = False, take = False, ext = 'jpg', augment = False):
    p = pathlib.Path(img_path)
    l = list(p.glob(str('**/*.' + ext)))
    image_names = [p.stem for p in l]
    
    if not take:
        print(take)
        preds = model.predict(dataset, verbose = 1)
        preds = np.ndarray.flatten(preds)

        print(preds.shape)
        if augment:
            preds = np.reshape(preds, (N_TEST, -1))
            print(preds.shape)
            preds = np.mean(preds, axis = 1)
        print(preds.shape)
        df = pd.DataFrame({'image_name': image_names, 
                           'target': preds})
        
    else:
        print(take)
        preds = model.predict(dataset.take(take), verbose = 1)
        print(preds.shape)
        preds = np.ndarray.flatten(preds)
        print(preds.shape)
        if augment:
            preds = np.reshape(preds, (N_TEST, -1))
            print(preds.shape)
            preds = np.mean(preds, axis = 1)
        print(preds.shape)
        df = pd.DataFrame({'image_name': image_names[:take*BATCH_SIZE], 
                           'target': preds})
    
    if isinstance(csv_filename, str):
        df.to_csv(pathlib.Path(SUBMITS_DIR, csv_filename), index = False)
    return df

#predicts = predict(model_4, DCM_TEST, ds_test, 'submit-22.csv', take = False, ext = 'dcm')

In [ ]:
preds_0 = predict(model_0, DCM_TEST, ds_test_augment, csv_filename = False, 
                  take = False, ext = 'dcm', augment = True)


In [ ]:
preds_4 = predict(model_4, DCM_TEST, ds_test, csv_filename = 'submit-30.csv', take = False, ext = 'dcm')